In [ ]:
!pip install mediapipe

# 1. Import

In [11]:
import cv2
import numpy as np
import os
from matplotlib import pyplot as plt
import time
import mediapipe as mp

# 2. Keypoints using Mediapipe Holistic

In [12]:
mp_holistic = mp.solutions.holistic # Holistic model
mp_drawing = mp.solutions.drawing_utils # drawing utils

In [13]:
def mediapipe_detection(image, model):
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB) 
    image.flags.writeable = False
    results = model. process(image)                 # Make prediction
    image.flags.writeable = True
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
    return image, results

In [14]:
def draw_landmarks(image, results):
    mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACEMESH_TESSELATION)
    mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS)    
    mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS)    
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS)    

In [15]:
def draw_styled_landmarks(image, results):
    # Draw face connections
    mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACEMESH_TESSELATION, 
                             mp_drawing.DrawingSpec(color=(80,110,10), thickness=1, circle_radius=1), 
                             mp_drawing.DrawingSpec(color=(80,256,121), thickness=1, circle_radius=1)
                             ) 
    # Draw pose connections
    mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS,
                             mp_drawing.DrawingSpec(color=(80,22,10), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(80,44,121), thickness=2, circle_radius=2)
                             ) 
    # Draw left hand connections
    mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                             mp_drawing.DrawingSpec(color=(121,22,76), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(121,44,250), thickness=2, circle_radius=2)
                             ) 
    # Draw right hand connections  
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                             mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2)
                             ) 

In [ ]:
# Test OpenCV video
cap = cv2.VideoCapture(0)
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    while cap.isOpened():
        # Read feed
        ret, frame = cap.read()

        # Make detections
        image, results = mediapipe_detection(frame, holistic)
        print(frame.shape)

        # Draw landmarks
        draw_styled_landmarks(image, results)

        # Show to screen
        cv2.imshow('OpenCV Feed', image)

        # Break
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break
    cap.release()
    cv2.destroyAllWindows()

In [ ]:
len(results.left_hand_landmarks.landmark)

In [ ]:
frame

In [ ]:
draw_styled_landmarks(frame,results)

In [ ]:
plt.imshow(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))

# 3. Extract Keypoints Values

In [16]:
def extract_keypoints(results):
    pose = np.array([[res.x, res.y, res.z, res.visibility] for res in results.pose_landmarks.landmark]).flatten() if results.pose_landmarks else np.zeros(33*4)
    face = np.array([[res.x, res.y, res.z] for res in results.face_landmarks.landmark]).flatten() if results.face_landmarks else np.zeros(468*3)
    lh = np.array([[res.x, res.y, res.z] for res in results.left_hand_landmarks.landmark]).flatten() if results.left_hand_landmarks else np.zeros(21*3)
    rh = np.array([[res.x, res.y, res.z] for res in results.right_hand_landmarks.landmark]).flatten() if results.right_hand_landmarks else np.zeros(21*3)
    return np.concatenate([pose, face, lh, rh])

In [ ]:
result_test = extract_keypoints(results)

In [ ]:
result_test

# 4. Setup Folders for Collection

In [17]:
# Path for exported data, numpy arrays
DATA_PATH = os.path.join('MP_Data_test') 

# Actions that we try to detect
actions = os.listdir(DATA_PATH)

# Thirty videos worth of data
no_sequences = 45

# Videos are going to be 45 frames in length
sequence_length = 45

In [ ]:
len(actions)

In [ ]:
os.makedirs('MP_Data',exist_ok=True)
for action in actions:
    os.makedirs(os.path.join(DATA_PATH, action),exist_ok=True)

In [ ]:
for action in actions: 
    for sequence in range(1,no_sequences+1):
        try: 
            os.makedirs(os.path.join(DATA_PATH, action, str(sequence)))
        except:
            pass

# 5. Collect Keypoint Valus for Training and Testing

In [ ]:
cap = cv2.VideoCapture(0)
# Set mediapipe model 
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    
    # NEW LOOP
    # Loop through actions
    for action in actions:
        # Loop through sequences aka videos
        for sequence in range(1,no_sequences+1):
            # Loop through video length aka sequence length
            for frame_num in range(sequence_length):

                # Read feed
                ret, frame = cap.read()

                # Make detections
                image, results = mediapipe_detection(frame, holistic)

                # Draw landmarks
                draw_styled_landmarks(image, results)
                
                # Apply wait logic
                if frame_num == 0: 
                    cv2.putText(image, 'STARTING COLLECTION', (120,200), 
                               cv2.FONT_HERSHEY_SIMPLEX, 1, (0,255, 0), 4, cv2.LINE_AA)
                    cv2.putText(image, 'Collecting frames for {} Video Number {}'.format(action, sequence), (15,12), 
                               cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1, cv2.LINE_AA)
                    # Show to screen
                    cv2.imshow('OpenCV Feed', image)
                    cv2.waitKey(500)
                else: 
                    cv2.putText(image, 'Collecting frames for {} Video Number {}'.format(action, sequence), (15,12), 
                               cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1, cv2.LINE_AA)
                    # Show to screen
                    cv2.imshow('OpenCV Feed', image)
                
                # Export keypoints
                keypoints = extract_keypoints(results)
                npy_path = os.path.join(DATA_PATH, action, str(sequence), str(frame_num))
                np.save(npy_path, keypoints)

                # Break gracefully
                if cv2.waitKey(10) & 0xFF == ord('q'):
                    break
                    
    cap.release()
    cv2.destroyAllWindows()

# 6.2 WLASL dataset
https://github.com/dxli94/WLASL

In [ ]:
root = 'C:\\Users\\ryanr\\ASLtoEng\\'
MP_DATA_PATH = os.path.join(root,'MP_data_test')

for label in os.listdir(MP_DATA_PATH):
    LABEL_PATH = os.path.join(MP_DATA_PATH,label)
    for filename in os.listdir(LABEL_PATH):
        if filename.endswith('.mp4'):
            print(f'Reading {filename}...')
            
            video_name = filename.split('.')[0]
            VIDEO_PATH = os.path.join(LABEL_PATH,filename)
            VIDEO_FOLDER_PATH = os.path.join(LABEL_PATH,video_name)
            os.makedirs(VIDEO_FOLDER_PATH,exist_ok=True)
            
            # Capture video
            vidcap = cv2.VideoCapture(VIDEO_PATH)
            # Set mediapipe model 
            with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:

                # Read feed
                success, frame = vidcap.read()
                
                frame_num = 0
                
                while success:
                    # Make detections
                    image, results = mediapipe_detection(frame, holistic)
                    
                    # Draw detections for debugging
#                     draw_styled_landmarks(image, results)
#                     cv2.imshow('OpenCV Feed', image)
#                     cv2.waitKey(10)                    

                    # Export keypoints
                    keypoints = extract_keypoints(results)
                    npy_path = os.path.join(VIDEO_FOLDER_PATH, str(frame_num))
                    np.save(npy_path, keypoints)
                         
                    success,frame = vidcap.read()
                    frame_num += 1
            
                    # Break gracefully
                    if cv2.waitKey(10) & 0xFF == ord('q'):
                        break
                
                print('Converted.')                    
                        
                # Break gracefully
                if cv2.waitKey(10) & 0xFF == ord('q'):
                    break
            
            # close the video
            vidcap.release()
            
cv2.destroyAllWindows()        

In [35]:
# remove videos after converting
for label in os.listdir(MP_DATA_PATH):
    LABEL_PATH = os.path.join(MP_DATA_PATH,label)
    for filename in os.listdir(LABEL_PATH):
        if filename.endswith('.mp4'):
            filepath = os.path.join(LABEL_PATH,filename)
            print(f'Deleting {filename}...')
            os.remove(filepath)

Deleting 00623.mp4...
Deleting 00624.mp4...
Deleting 00625.mp4...
Deleting 00626.mp4...
Deleting 00627.mp4...
Deleting 00628.mp4...
Deleting 00629.mp4...
Deleting 00631.mp4...
Deleting 00632.mp4...
Deleting 00633.mp4...
Deleting 00634.mp4...
Deleting 00639.mp4...
Deleting 01383.mp4...
Deleting 01384.mp4...
Deleting 01385.mp4...
Deleting 01386.mp4...
Deleting 01387.mp4...
Deleting 01388.mp4...
Deleting 01391.mp4...
Deleting 01398.mp4...
Deleting 65029.mp4...
Deleting 05229.mp4...
Deleting 05230.mp4...
Deleting 05231.mp4...
Deleting 05232.mp4...
Deleting 05233.mp4...
Deleting 05234.mp4...
Deleting 05238.mp4...
Deleting 05239.mp4...
Deleting 05243.mp4...
Deleting 65145.mp4...
Deleting 69225.mp4...


In [ ]:
# sanity check that the data collected is correct
for label in os.listdir(MP_DATA_PATH):
    LABEL_PATH = os.path.join(MP_DATA_PATH,label)
    for filename in os.listdir(LABEL_PATH):
        if not filename.endswith('.mp4'):
            for npfile in os.listdir(os.path.join(LABEL_PATH,filename)):
                print(np.load(os.path.join(LABEL_PATH,filename,npfile)))

# 6.3 ASLLRP dataset 
http://www.bu.edu/asllrp/av/dai-asllvd.html

# 6.4 ASL MNIST dataset
https://www.kaggle.com/datamunge/sign-language-mnist

# 6.* Preprocess Data and Create Labels and Features

In [28]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical

In [18]:
label_map = {label:num for num, label in enumerate(actions)}
print(label_map)

{'accident': 0, 'africa': 1, 'basketball': 2}


In [ ]:
# os.chdir('..')
os.listdir()

In [ ]:
for action in actions:
    for sequence in np.array(os.listdir(os.path.join(DATA_PATH, action))):
        for frame_num in range(sequence_length):
            if  os.path.exists(os.path.join(DATA_PATH, action, sequence, "{}.npy".format(frame_num))):
                res = np.load(os.path.join(DATA_PATH, action, sequence, "{}.npy".format(frame_num)))d
            else:
                res = np.zeros((1662,),dtype='float64')
            # TODO: plot recognized frame
            break
        break
    break
    

In [37]:
sequences, labels = [], []
for action in label_map.keys():
    print(np.array(os.listdir(os.path.join(DATA_PATH, action))))
    for sequence in np.array(os.listdir(os.path.join(DATA_PATH, action))):
        window = []
        for frame_num in range(sequence_length):
            if  os.path.exists(os.path.join(DATA_PATH, action, sequence, "{}.npy".format(frame_num))):
                res = np.load(os.path.join(DATA_PATH, action, sequence, "{}.npy".format(frame_num)))
            else:
                res = np.zeros((1662,),dtype='float64')
            window.append(res)
        sequences.append(window)
        labels.append(label_map[action])

['00623' '00624' '00625' '00626' '00627' '00628' '00629' '00631' '00632'
 '00633' '00634' '00639']
['01383' '01384' '01385' '01386' '01387' '01388' '01391' '01398' '65029']
['05229' '05230' '05231' '05232' '05233' '05234' '05238' '05239' '05243'
 '65145' '69225']


In [ ]:
sequences

In [ ]:
labels

In [40]:
X = np.array(sequences)

In [41]:
X.shape

(32, 45, 1662)

In [52]:
y.shape

(32, 3)

In [42]:
# y = to_categorical(labels,num_classes=len(actions)).astype(int)
y = to_categorical(labels).astype(int)

In [47]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1)

In [48]:
np.argmax(y_test, axis=1).tolist()

[1, 0, 2, 0]

# 7. Build and Train LSTM Neural Network

In [2]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.callbacks import TensorBoard

In [ ]:
log_dir = os.path.join('Logs')
tb_callback = TensorBoard(log_dir=log_dir)

In [5]:
model = Sequential()
model.add(LSTM(64, return_sequences=True, activation='relu', input_shape=(45,1662)))
model.add(LSTM(128, return_sequences=True, activation='relu'))
model.add(LSTM(64, return_sequences=False, activation='relu'))
model.add(Dense(128, activation='relu'))
model.add(Dense(256, activation='relu'))
model.add(Dense(y.shape[1], activation='softmax'))

In [6]:
model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['categorical_accuracy'])

In [54]:
model.fit(X_train, y_train, epochs=200, callbacks=[tb_callback])

Epoch 1/200
1/1 [==============================] - 7s 7s/step - loss: 1.0983 - categorical_accuracy: 0.2857
Epoch 2/200
1/1 [==============================] - 0s 466ms/step - loss: 5.8979 - categorical_accuracy: 0.3571
Epoch 3/200
1/1 [==============================] - 0s 473ms/step - loss: 37.7889 - categorical_accuracy: 0.4643
Epoch 4/200
1/1 [==============================] - 0s 442ms/step - loss: 19.8545 - categorical_accuracy: 0.3571
Epoch 5/200
1/1 [==============================] - 0s 491ms/step - loss: 15.2024 - categorical_accuracy: 0.2857
Epoch 6/200
1/1 [==============================] - 0s 451ms/step - loss: 13.1622 - categorical_accuracy: 0.2857
Epoch 7/200
1/1 [==============================] - 1s 506ms/step - loss: 132.1334 - categorical_accuracy: 0.3571
Epoch 8/200
1/1 [==============================] - 1s 823ms/step - loss: 33.9515 - categorical_accuracy: 0.3214
Epoch 9/200
1/1 [==============================] - 1s 616ms/step - loss: 19.9769 - categorical_accuracy: 0.3

1/1 [==============================] - 0s 323ms/step - loss: 113.3567 - categorical_accuracy: 0.2857
Epoch 74/200
1/1 [==============================] - 0s 371ms/step - loss: 66.7558 - categorical_accuracy: 0.4286
Epoch 75/200
1/1 [==============================] - 0s 360ms/step - loss: 129.8331 - categorical_accuracy: 0.4286
Epoch 76/200
1/1 [==============================] - 0s 353ms/step - loss: 104.3977 - categorical_accuracy: 0.1786
Epoch 77/200
1/1 [==============================] - 0s 322ms/step - loss: 77.6211 - categorical_accuracy: 0.3571
Epoch 78/200
1/1 [==============================] - 0s 330ms/step - loss: 94.6825 - categorical_accuracy: 0.2500
Epoch 79/200
1/1 [==============================] - 0s 363ms/step - loss: 82.3800 - categorical_accuracy: 0.2857
Epoch 80/200
1/1 [==============================] - 0s 388ms/step - loss: 41.7863 - categorical_accuracy: 0.5000
Epoch 81/200
1/1 [==============================] - 0s 345ms/step - loss: 107.4202 - categorical_accuracy:

Epoch 145/200
1/1 [==============================] - 0s 369ms/step - loss: 41.6406 - categorical_accuracy: 0.3214
Epoch 146/200
1/1 [==============================] - 0s 356ms/step - loss: 37.9077 - categorical_accuracy: 0.4643
Epoch 147/200
1/1 [==============================] - 0s 360ms/step - loss: 19.8810 - categorical_accuracy: 0.5714
Epoch 148/200
1/1 [==============================] - 1s 579ms/step - loss: 53.8857 - categorical_accuracy: 0.3571
Epoch 149/200
1/1 [==============================] - 0s 458ms/step - loss: 45.1396 - categorical_accuracy: 0.4286
Epoch 150/200
1/1 [==============================] - 0s 352ms/step - loss: 44.5343 - categorical_accuracy: 0.3214
Epoch 151/200
1/1 [==============================] - 0s 387ms/step - loss: 55.3104 - categorical_accuracy: 0.2143
Epoch 152/200
1/1 [==============================] - 0s 361ms/step - loss: 31.6161 - categorical_accuracy: 0.3214
Epoch 153/200
1/1 [==============================] - 0s 341ms/step - loss: 34.1917 - cat

In [ ]:
model.summary()

# 8. Make Predictions

In [55]:
res = model.predict(X_test)
res.shape

(4, 3)

In [56]:
actions[np.argmax(res[2])]

'basketball'

In [57]:
actions[np.argmax(y_test[2])]

'basketball'

# 9. Save Weights

In [58]:
model.save('action.h5')

In [59]:
del model

In [7]:
model.load_weights('action.h5')

# 10. Evaluation using Confusion Matrix and Accuracy

In [64]:
from sklearn.metrics import multilabel_confusion_matrix, accuracy_score

In [65]:
yhat = model.predict(X_test)

In [66]:
len(y_test)

4

In [67]:
yhat

array([[9.9980348e-01, 4.8373937e-07, 1.9605504e-04],
       [9.9999988e-01, 7.8965980e-09, 7.5181660e-08],
       [6.3179806e-07, 1.7409593e-09, 9.9999940e-01],
       [4.1848630e-01, 2.3294869e-01, 3.4856501e-01]], dtype=float32)

In [73]:
ytrue = np.argmax(y_test, axis=1).tolist()
yhat = np.argmax(yhat, axis=1).tolist()

In [74]:
ytrue

[1, 0, 2, 0]

In [75]:
yhat

[0, 0, 2, 0]

In [76]:
multilabel_confusion_matrix(ytrue, yhat)

array([[[1, 1],
        [0, 2]],

       [[3, 0],
        [1, 0]],

       [[3, 0],
        [0, 1]]], dtype=int64)

In [77]:
accuracy_score(ytrue, yhat)

0.75

# 11. Test in Real Time

In [8]:
from scipy import stats

In [9]:
colors = [(245,117,16), (117,245,16), (16,117,245)]
def prob_viz(res, actions, input_frame, colors):
    output_frame = input_frame.copy()
    for num, prob in enumerate(res):
        cv2.rectangle(output_frame, (0,60+num*40), (int(prob*100), 90+num*40), colors[num], -1)
        cv2.putText(output_frame, actions[num], (0, 85+num*40), cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,255), 2, cv2.LINE_AA)
        
    return output_frame

In [ ]:
plt.figure(figsize=(18,18))
plt.imshow(prob_viz(res, actions, image, colors))

In [19]:
# 1. New detection variables
sequence = []
sentence = []
predictions = []
threshold = 0.5

cap = cv2.VideoCapture(0)
# Set mediapipe model 
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    while cap.isOpened():

        # Read feed
        ret, frame = cap.read()

        # Make detections
        image, results = mediapipe_detection(frame, holistic)
        # print(results)
        
        # Draw landmarks
        draw_styled_landmarks(image, results)
        
        # 2. Prediction logic
        keypoints = extract_keypoints(results)
        sequence.append(keypoints)
        sequence = sequence[-no_sequences:]
        
        if len(sequence) == no_sequences:
            res = model.predict(np.expand_dims(sequence, axis=0))[0]
            print(actions[np.argmax(res)])
            predictions.append(np.argmax(res))
            
            
        #3. Viz logic
            if np.unique(predictions[-10:])[0]==np.argmax(res): 
                if res[np.argmax(res)] > threshold: 
                    
                    if len(sentence) > 0: 
                        if actions[np.argmax(res)] != sentence[-1]:
                            sentence.append(actions[np.argmax(res)])
                    else:
                        sentence.append(actions[np.argmax(res)])

            if len(sentence) > 5: 
                sentence = sentence[-5:]

            # Viz probabilities
            image = prob_viz(res, actions, image, colors)
            
        cv2.rectangle(image, (0,0), (640, 40), (245, 117, 16), -1)
        cv2.putText(image, ' '.join(sentence), (3,30), 
                       cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
        
        # Show to screen
        cv2.imshow('OpenCV Feed', image)

        # Break gracefully
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break
    cap.release()
    cv2.destroyAllWindows()

africa
accident
accident
basketball
accident
accident
accident
accident
accident
africa
basketball
basketball
accident
accident
accident
accident
accident
accident
accident
basketball
accident
accident
accident
accident
basketball
accident
accident
africa
accident
basketball
africa
accident
africa
basketball
basketball
basketball
africa
accident
accident
accident
accident
accident
basketball
accident
basketball
africa
accident
africa
basketball
accident
accident
accident
accident
africa
accident
accident
africa
basketball
accident
accident
accident
africa
accident
accident
accident
accident
accident
africa
africa
accident
accident
basketball
basketball
accident
accident
basketball
africa
basketball
accident
africa
africa
accident
basketball
africa
accident
accident
accident
accident
africa
accident
accident
africa
accident
basketball
accident
accident
accident
accident
accident
accident
accident
accident
accident
accident
basketball
accident
accident
accident
basketball
accident
accide